In [69]:
import os, sys
import pandas as pd
import numpy as np
import random as rd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings('ignore')

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
cf.go_offline()
from pandas_profiling import ProfileReport

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,OneHotEncoder,StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_selection import RFE

from numpy import set_printoptions
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB



In [70]:
exps_dir = "../../exps"
if os.path.exists(exps_dir) == False: # tạo thư mục (nếu chưa có)
  os.makedirs(exps_dir, exist_ok=True)

save_dir = f"{exps_dir}/feature1"
os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

* kiểm tra và tạo các thư mục (nếu chưa có)
* tập test 33%

# 5. Xây dựng và đánh giá mô hình học sâu Neutual Network

In [71]:
x_train=pd.read_excel(f'{save_dir}/x_train.xlsx')
y_train=pd.read_excel(f'{save_dir}/y_train.xlsx')
x_test=pd.read_excel(f'{save_dir}/x_test.xlsx')
y_test=pd.read_excel(f'{save_dir}/y_test.xlsx')
class_weights_dict=dict(np.load(f'{exps_dir}/feature1/class_weights_dict.npz',allow_pickle=True))['class_weights_dict']
class_weights_dict = {key: value for key, value in class_weights_dict.item().items()}
class_weights_dict

{0: 1.0, 1: 1.0}

* Sử dụng thư viện **imbalanced-learn** để thực hiện Oversampling bằng phương pháp **SMOTE (Synthetic Minority Over-sampling Technique).** 

#### Giải thích:
* Vì tập dữ liệu của nhóm là tập dữ liệu phân loại không cân bằng => **SMOTE** được sử dụng để tăng cường dữ liệu trong trường hợp mẫu của lớp thiểu số (minority class) quá ít so với lớp đa số (majority class), giúp cân bằng dữ liệu và cải thiện hiệu suất của mô hình.

In [72]:
x_test.shape

(330, 42)

In [73]:
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

# Lớp thứ hai
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

# Lớp thứ ba
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

# Lớp thứ tư
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

# Lớp đầu ra với activation function 'sigmoid' cho bài toán phân lớp nhị phân
model.add(Dense(1, activation='sigmoid'))

model.summary()



In [ ]:
plot_model(model, to_file="model.png", show_shapes=True)

### Giải thích:
   Nhóm sử dụng thư viện TensorFlow và Keras xây dựng một mô hình mạng nơ-ron sử dụng Sequential API:
   
   1. Đầu tiên, tạo mô hình Sử dụng mô hình tuần tự **Sequential**
   2. Sau đó, thêm các lớp Dense với hàm kích hoạt **'relu'**. Nhóm sử dụng **Dropout** tại mỗi tầng để ngẫu nhiên "tắt" khoảng 50% số nơ-ron tại tầng đó trong quá trình huấn luyện, ngăn chặn hiện tượng quá mức học (overfitting).
   3. Thêm lớp output với hàm kích hoạt 'tanh'. Hàm **'tanh'** chuyển đổi mỗi giá trị đầu ra từ khoảng (-∞, ∞) về khoảng (-1, 1).
   4. Compile mô hình: 
      * loss=**binary_crossentropy** được sử dụng vì đây là một bài toán phân loại nhị phân. 
      * optimizer=**adam** là một thuật toán tối ưu hóa phổ biến. 
      * metric=**'accuracy'**, Theo dõi độ chính xác của mô hình trong quá trình huấn luyện.

In [74]:
lr=0.0001
model.compile(optimizer=Adam(lr=lr), loss='binary_crossentropy', metrics='accuracy')
earlystopping = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 30)

early_stopping = EarlyStopping(monitor='val_accuracy',
    min_delta=0.00005,
    patience=11,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=30,
    min_lr=0.000001,
    verbose=1,
)
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import confusion_matrix

class ConfusionMatrixCallback(Callback):
    def __init__(self, validation_data):
        self.validation_data = validation_data
        self.relist={}
        self.acc=[]

    def on_epoch_end(self, epoch, logs=None):
        X_val, y_val = self.validation_data
        y_pred = self.model.predict(X_val)
        y_pred_binary = (y_pred > 0.5).astype(int)

        cm = confusion_matrix(y_val, y_pred_binary)
        print(f"Confusion Matrix after Epoch {epoch + 1}:\n{cm}")
        print(classification_report(y_test,y_pred_binary))
        acc = float(classification_report(y_test,y_pred_binary).split('\n')[3].split()[2])
        self.acc.append(acc)
        self.relist[acc]=y_pred_binary

# ... Định nghĩa mô hình ...

# Tạo đối tượng callback
confusion_matrix_callback = ConfusionMatrixCallback(validation_data=(x_test,y_test))

callbacks = [early_stopping,lr_scheduler]

checkpointer = ModelCheckpoint(filepath = "Emotion_weights.hdf5", verbose = 1, save_best_only=True)

history=model.fit(x=x_train,y=y_train,
          validation_data=(x_test,y_test),class_weight=class_weights_dict ,
          batch_size=64,epochs=120, callbacks=[confusion_matrix_callback])



Epoch 1/120
11/11 [==============================] - 0s 3ms/steposs: 1.0573 - accuracy: 0.50
Confusion Matrix after Epoch 1:
[[239   3]
 [ 88   0]]
              precision    recall  f1-score   support

           0       0.73      0.99      0.84       242
           1       0.00      0.00      0.00        88

    accuracy                           0.72       330
   macro avg       0.37      0.49      0.42       330
weighted avg       0.54      0.72      0.62       330

16/16 [==============================] - 3s 60ms/step - loss: 1.0363 - accuracy: 0.5020 - val_loss: 0.6001 - val_accuracy: 0.7242
Epoch 2/120
11/11 [==============================] - 0s 2ms/steposs: 0.9803 - accuracy: 0.54
Confusion Matrix after Epoch 2:
[[242   0]
 [ 86   2]]
              precision    recall  f1-score   support

           0       0.74      1.00      0.85       242
           1       1.00      0.02      0.04        88

    accuracy                           0.74       330
   macro avg       0.87      

In [75]:
print(confusion_matrix_callback.acc)
print(confusion_matrix_callback.relist)

[0.0, 0.02, 0.06, 0.11, 0.19, 0.27, 0.3, 0.39, 0.43, 0.49, 0.52, 0.55, 0.57, 0.59, 0.58, 0.62, 0.66, 0.62, 0.65, 0.68, 0.68, 0.68, 0.66, 0.64, 0.64, 0.59, 0.59, 0.61, 0.58, 0.57, 0.58, 0.58, 0.58, 0.61, 0.6, 0.59, 0.6, 0.6, 0.59, 0.56, 0.55, 0.55, 0.59, 0.59, 0.6, 0.59, 0.6, 0.6, 0.59, 0.6, 0.6, 0.6, 0.59, 0.59, 0.62, 0.61, 0.6, 0.57, 0.58, 0.59, 0.6, 0.59, 0.59, 0.59, 0.59, 0.59, 0.58, 0.57, 0.57, 0.57, 0.59, 0.57, 0.56, 0.57, 0.56, 0.59, 0.57, 0.57, 0.56, 0.57, 0.56, 0.56, 0.57, 0.56, 0.57, 0.56, 0.56, 0.57, 0.56, 0.53, 0.55, 0.55, 0.56, 0.55, 0.56, 0.57, 0.55, 0.57, 0.57, 0.57, 0.55, 0.56, 0.56, 0.56, 0.55, 0.53, 0.55, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.51, 0.52, 0.52, 0.55, 0.56, 0.55, 0.56]
{0.0: array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]

In [76]:
predictions=confusion_matrix_callback.relist[max(confusion_matrix_callback.acc)]
predictions
max(confusion_matrix_callback.acc)

0.68

In [93]:
(classification_report(y_test,predictions)).split('\n')[5]

'    accuracy                           0.57       330'

In [77]:
for epoch, acc in enumerate(history.history['val_accuracy']):
    print(f"Epoch {epoch + 1} - Validation Accuracy: {acc}")

Epoch 1 - Validation Accuracy: 0.7242424488067627
Epoch 2 - Validation Accuracy: 0.739393949508667
Epoch 3 - Validation Accuracy: 0.7272727489471436
Epoch 4 - Validation Accuracy: 0.7030302882194519
Epoch 5 - Validation Accuracy: 0.699999988079071
Epoch 6 - Validation Accuracy: 0.6666666865348816
Epoch 7 - Validation Accuracy: 0.6424242258071899
Epoch 8 - Validation Accuracy: 0.6212121248245239
Epoch 9 - Validation Accuracy: 0.6000000238418579
Epoch 10 - Validation Accuracy: 0.6121212244033813
Epoch 11 - Validation Accuracy: 0.5787878632545471
Epoch 12 - Validation Accuracy: 0.5757575631141663
Epoch 13 - Validation Accuracy: 0.5757575631141663
Epoch 14 - Validation Accuracy: 0.5727272629737854
Epoch 15 - Validation Accuracy: 0.5515151619911194
Epoch 16 - Validation Accuracy: 0.560606062412262
Epoch 17 - Validation Accuracy: 0.5545454621315002
Epoch 18 - Validation Accuracy: 0.5545454621315002
Epoch 19 - Validation Accuracy: 0.5696969628334045
Epoch 20 - Validation Accuracy: 0.560606062

### Note:
   #### 1. Early Stopping:
   * **monitor='val_accuracy'**: Đây là đại lượng mà quá trình Early Stopping sẽ theo dõi. Trong trường hợp này, nó sẽ kiểm tra độ chính xác trên tập validation.
   * **min_delta=0.00005**: Số lượng thay đổi nhỏ nhất giữa hai epoch để coi là có sự cải thiện. Nếu không có sự cải thiện lớn hơn min_delta sau số lượng epoch 'patience', quá trình huấn luyện sẽ dừng lại.
   * **patience=11**: Số epoch mà mô hình có thể không cải thiện trước khi quá trình Early Stopping được kích hoạt.
   * **verbose=1**: Hiển thị thông báo khi quá trình Early Stopping được kích hoạt.
   * **restore_best_weights=True**: Khôi phục trọng số của mô hình tại epoch có độ chính xác cao nhất trên tập validation.
    
   #### 2. Learning Rate Scheduler (ReduceLROnPlateau):
   * **monitor='val_accuracy'**: Đại lượng mà Learning Rate Scheduler sẽ theo dõi, cũng là độ chính xác trên tập validation.
   * **factor=0.5**: Hệ số giảm learning rate. Khi được kích hoạt (sau 'patience' epochs mà không có cải thiện), learning rate sẽ được giảm đi một lượng bằng factor.
   * **patience=7**: Số epoch mà mô hình có thể không cải thiện trước khi learning rate được giảm.
   * **min_lr=0.00001**: Giới hạn dưới cho learning rate sau khi được giảm. Không bao giờ giảm dưới giá trị này.
   * **verbose=1**: Hiển thị thông báo khi quá trình giảm learning rate được kích hoạt.

### Giải thích:
   Nhóm thực hiện quá trình huấn luyện mô hình máy học với **model.fit**:
   
   * **x=x_train, y=y_train:** Dữ liệu đầu vào và nhãn cho quá trình đào tạo.
   * **validation_data=(x_test,y_test):** Dữ liệu kiểm tra.
   * **class_weight=class_weights_dict:** Trọng số lớp được áp dụng cho cân bằng lớp. (-> Vì tập dữ liệu mất cân bằng lớp)
   * Kích thước batch là **batch_size=128.** 
   * Số lượng epochs là **epochs=600.** 
   * Nhóm sử dụng **callbacks = [early_stopping,lr_scheduler]** để đảm bảo rằng mô hình được lưu lại tại những điểm có độ chính xác tốt nhất trên tập kiểm tra và đồng thời ngừng sớm nếu không có sự cải thiện đáng kể. Bên cạnh đó, giảm learning rate giúp mô hình hội tụ chậm hơn và tránh việc vượt qua điểm tối ưu toàn cục.

<AxesSubplot: >